**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/ParslFest_2025/01_sequential.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
import os
import re
from util import setup_environment, run_make_command, parse_total_time

# Configuration flags: set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = False  

print(f"{os.getcwd() = }")
output_dir = setup_environment()

if force_compute:  # Clean local only files if force_compute is set
    !(cd ../../.. && make clean_local_only_files)
    !rm -fr output/*.* runinfo/  # remove output directory contents

os.getcwd() = '/Users/terrya/Documents/ProgramData/QMCSoftware_resume/demos/talk_paper_demos/ParslFest_2025'
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
		rm -f "$f"; > /dev/null 2>&1; \
	done


## 1. Sequential Execution

In [2]:
out_path = os.path.join(output_dir, "sequential_output.csv")
if (not os.path.exists(out_path)) or force_compute:
    run_make_command("booktests_no_docker", out_path, is_debug=is_debug)

True

In [3]:
import pandas as pd

seq_path = os.path.join(output_dir, "sequential_output.csv")
sequential_time = parse_total_time(seq_path)

if sequential_time > 0:  # save time to sequential_time.csv
    seq_time_path = os.path.join(output_dir, "sequential_time.csv")
    with open(seq_time_path, "w") as f:
        _ = f.write(f"{sequential_time:.2f}\n")
    print(f"Sequential time: {sequential_time:.2f} seconds")
else:
    print("Warning: Could not parse sequential time from output")

# Parse individual test memory and time
with open(seq_path, "r") as f:
    text = f.read()
test_pattern = re.compile(r"(test_\w+)\s+\([^)]+\)\s+\.\.\.\s+Memory used:\s*([\d\.]+)\s*GB\.\s*Test time:\s*([\d\.]+)\s*s")
test_matches = test_pattern.findall(text)

if test_matches:  # Write time data to CSV files
    seq_time_detail_path = os.path.join(output_dir, "sequential_output_time.csv")
    rows = [{"Notebook": t[0].replace("_notebook", ""), "Memory_GB": float(t[1]), "Time_s": float(t[2])} 
            for t in test_matches]
    df = pd.DataFrame(rows).sort_values(by="Time_s", ascending=False)
    df.to_csv(seq_time_detail_path, index=False)
    print(f"Parsed {len(test_matches)} test results")
else:
    print("Warning: Could not parse individual test results")

Sequential time: 482.21 seconds
Parsed 26 test results


In [4]:
# free memory
import gc
gc.collect();